In [1]:
import requests
import pandas as pd

In [52]:
api_key = 'Y1wqSBHqhkOrzj0Ln0ZXQVDEXbFzTXb2'

In [53]:
tickers = ['AAPL', 'NVDA', 'WMT']

In [54]:
# sim_ids = []

for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()
    print(data)

[{'simId': 111052, 'ticker': 'AAPL', 'name': 'APPLE INC'}]
[{'simId': 172199, 'ticker': 'NVDA', 'name': 'NVIDIA CORP'}]
[{'simId': 239962, 'ticker': 'WMT', 'name': 'WAL MART STORES INC'}]


In [56]:
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

[111052, 172199, 239962]


In [7]:
# define time periods for financial statement data
statement_type = "pl"
time_periods = ["Q1","Q2","Q3","Q4"]
year_start = 2013
year_end = 2018

In [8]:
data = {}
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]

In [14]:
data['AAPL']

{'Line Item': ['Revenue',
  'Sales & Services Revenue',
  'Financing Revenue',
  'Other Revenue',
  'Cost of revenue',
  'Cost of Goods & Services',
  'Cost of Financing Revenue',
  'Cost of Other Revenue',
  'Gross Profit',
  'Other Operating Income',
  'Operating Expenses',
  'Selling, General & Administrative',
  'Selling & Marketing',
  'General & Administrative',
  'Research & Development',
  'Depreciation & Amortization',
  'Provision For Doubtful Accounts',
  'Other Operating Expense',
  'Operating Income (Loss)',
  'Non-Operating Income (Loss)',
  'Interest Expense, net',
  'Interest Expense',
  'Interest Income',
  'Other Investment Income (Loss)',
  'Foreign Exchange Gain (Loss)',
  'Income (Loss) from Affiliates',
  'Other Non-Operating Income (Loss)',
  'Pretax Income (Loss), Adjusted',
  'Abnormal Gains (Losses)',
  'Acquired In-Process R&D',
  'Merger / Acquisition Expense',
  'Abnormal Derivatives',
  'Disposal of Assets',
  'Early extinguishment of Debt',
  'Asset Write

In [11]:
df = pd.DataFrame(data=d)

In [12]:
df

,Line Item,Q1-2013,Q2-2013,Q3-2013,Q4-2013,Q1-2014,Q2-2014,Q3-2014,Q4-2014,Q1-2015,...,Q3-2016,Q4-2016,Q1-2017,Q2-2017,Q3-2017,Q4-2017,Q1-2018,Q2-2018,Q3-2018,Q4-2018
0,Revenue,114187000000,116830000000,115688000000,1.295890e+11,114960000000,120125000000,119001000000,1.315650e+11,114826000000,...,118179000000,1.309360e+11,117542000000,123355000000,123179000000,1.362670e+11,122690000000,128028000000,1.248940e+11,None
1,Sales & Services Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
2,Financing Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
3,Other Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
4,Cost of revenue,-86027000000,-87420000000,-86687000000,-9.793500e+10,-86714000000,-90010000000,-89247000000,-9.911500e+10,-86483000000,...,-87484000000,-9.774300e+10,-87688000000,-91521000000,-91547000000,-1.026400e+11,-91707000000,-95571000000,-9.311600e+10,None
5,Cost of Goods & Services,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
6,Cost of Financing Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
7,Cost of Other Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None
8,Gross Profit,28160000000,29410000000,29001000000,3.165400e+10,28246000000,30115000000,29754000000,3.245000e+10,28343000000,...,30695000000,3.319300e+10,29854000000,31834000000,31632000000,3.362700e+10,30983000000,32457000000,3.177800e+10,None
9,Other Operating Income,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,NaN,None


In [60]:
company_id = '111052'
url = 'https://simfin.com/api/v1/companies/id/' + company_id + '/ratios?api-key=Y1wqSBHqhkOrzj0Ln0ZXQVDEXbFzTXb2'
content = requests.get(url)
content = content.json()
# https://simfin.com/api/v1/companies/id/{companyId}/ratios

In [61]:
content

[{'indicatorId': '0-31',
  'indicatorName': 'Last Closing Price',
  'value': '189.95',
  'period': None,
  'fyear': None,
  'currency': 'USD',
  'period-end-date': '2019-03-29'},
 {'indicatorId': '0-64',
  'indicatorName': 'Common Shares Outstanding',
  'value': '4715280000',
  'period': None,
  'fyear': None,
  'currency': None,
  'period-end-date': '2019-01-18'},
 {'indicatorId': '0-65',
  'indicatorName': 'Preferred Shares Outstanding',
  'value': None,
  'period': None,
  'fyear': None,
  'currency': None,
  'period-end-date': None},
 {'indicatorId': '0-66',
  'indicatorName': 'Average Shares Outstanding, basic',
  'value': '4861112750',
  'period': 'TTM',
  'fyear': 2019,
  'currency': None,
  'period-end-date': '2018-12-31'},
 {'indicatorId': '0-67',
  'indicatorName': 'Average Shares Outstanding, diluted',
  'value': '4903975250',
  'period': 'TTM',
  'fyear': 2019,
  'currency': None,
  'period-end-date': '2018-12-31'},
 {'indicatorId': '1-1',
  'indicatorName': 'Revenues',
  '

In [62]:
pd.options.display.max_rows = 100

In [63]:
df2 = pd.DataFrame(data=content)

In [64]:
df2

,currency,fyear,indicatorId,indicatorName,period,period-end-date,value
0,USD,NaN,0-31,Last Closing Price,None,2019-03-29,189.95
1,None,NaN,0-64,Common Shares Outstanding,None,2019-01-18,4715280000
2,None,NaN,0-65,Preferred Shares Outstanding,None,None,None
3,None,2019.0,0-66,"Average Shares Outstanding, basic",TTM,2018-12-31,4861112750
4,None,2019.0,0-67,"Average Shares Outstanding, diluted",TTM,2018-12-31,4903975250
5,USD,2019.0,1-1,Revenues,TTM,2018-12-31,261612000000
6,USD,2019.0,1-2,Cost of Goods Sold,TTM,2018-12-31,161654000000
7,USD,2019.0,1-4,Gross Profit,TTM,2018-12-31,99958000000
8,USD,2019.0,1-11,Operating Expenses,TTM,2018-12-31,31988000000
9,USD,2019.0,1-12,"Selling, General and Administrative",TTM,2018-12-31,17257000000


In [98]:
for index, row in df2.iterrows():
    print (row['currency'])

USD
None
None
None
None
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [44]:
import requests, csv, datetime
import pandas as pd
from csv import writer, reader


 
def top_50_by_comments():      # gets top 50 ticker symbols and returns a list

    all_comments = pd.read_csv('all-comments.csv')
    begin_date = datetime.date.today() + datetime.timedelta(-30)  # gets 30 day prior date
    begin_date = begin_date.strftime('%Y-%m-%d')    # converts to a string
    recent_comments = all_comments[all_comments['date'] > begin_date]
    recent_comments = recent_comments['ticker'].value_counts().head(50).index.tolist()
    return recent_comments
    
    
def sim_ids():   # get sim_ids from SimFin from tickers.  Calls the top_50_by_comments func
    
    tickers = top_50_by_comments()
    api_key = 'Y1wqSBHqhkOrzj0Ln0ZXQVDEXbFzTXb2'
    sim_ids = []    
    for ticker in tickers:
        request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
        content = requests.get(request_url)
        data = content.json()

        if "error" in data or len(data) < 1:
            sim_ids.append(None)
        else:
            sim_ids.append(data[0]['simId'])
    return sim_ids

    
    
def fin_ratios():

    api_key = 'Y1wqSBHqhkOrzj0Ln0ZXQVDEXbFzTXb2'
    tickers = top_50_by_comments()
    ticker_counter = 0
    company_ids = sim_ids()

#     tickers = ['XRX', 'ORCL', 'WIN', 'WMT', 'SHLD', 'VZ', 'GE', 'SAP', 'BK', 'UNP']
#     ticker_counter = 0
#     company_ids = [378247, 257930, 625612, 239962, 100266, 101219, 244314, None, 467599, 367644]

    
    with open('fin_ratios.csv', 'w', encoding='utf-8', newline = '') as file:    #Opens new file to write to
        csv_object = csv.writer(file)
        csv_object.writerow(['ticker', 'company_id', 'currency', 'fyear', 'indicatorId',
                            'indicatorName', 'period', 'period_end_date', 'value'])  #Header

        for company in company_ids:  # loops through each of the top tickers
            try:
                url = f'https://simfin.com/api/v1/companies/id/{company}/ratios?api-key=Y1wqSBHqhkOrzj0Ln0ZXQVDEXbFzTXb2'
                content = requests.get(url)
                content = content.json()
                df = pd.DataFrame(data=content) #puts into a dataframe

                for index, row in df.iterrows():   # goes through each row and writes to csv file
                    ticker = tickers[ticker_counter]
                    company_id = company
                    currency = row['currency']
                    fyear = row['fyear']
                    indicatorId = row['indicatorId']
                    indicatorName = row['indicatorName']
                    period = row['period']
                    period_end_date = row['period-end-date']
                    value = row['value']
                    csv_object.writerow([ticker, company_id, currency, fyear, indicatorId, 
                                        indicatorName, period, period_end_date, value])
            except:
                pass
            ticker_counter += 1
    

In [43]:
fin_ratios()